# Backtesting ML Classification-Based

In [ ]:
???

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],99.873096
Equity Final [$],45536.090993
Equity Peak [$],59377.806997
Return [%],355.36091


## Load the model

In [6]:
import pickle

In [7]:
with open('models/model_dt_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

## Load the data

In [ ]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

## Backtesting.py Library

### Create your Strategy Class

In [11]:
from backtesting import ???

In [ ]:
Backtest(???)

In [40]:
class ???(Strategy):
    def init(self):
        self.model = ???

    def next(self):
        explanatory_today = ???
        forecast_tomorrow = ???
        
        # conditions to sell or buy

#### Calculate Purchase Recommendation

##### Buy if it goes up

In [43]:
df[['change_tomorrow_direction']].head(10)

,change_tomorrow_direction,buy_sell
Date,,
2016-12-08,UP,1
2016-12-09,UP,1
2016-12-12,UP,1
2016-12-13,DOWN,-1
2016-12-14,DOWN,-1
2016-12-15,DOWN,-1
2016-12-16,UP,1
2016-12-19,DOWN,-1
2016-12-20,DOWN,-1


##### Buy if it goes and sell if down

> You can only sell if you have already bought

In [51]:
df[['change_tomorrow_direction', 'buy_sell']].head(10)

,change_tomorrow_direction,buy_sell,buy_sell_track
Date,,,
2016-12-08,UP,1,1
2016-12-09,UP,1,0
2016-12-12,UP,1,0
2016-12-13,DOWN,-1,-1
2016-12-14,DOWN,-1,0
2016-12-15,DOWN,-1,0
2016-12-16,UP,1,1
2016-12-19,DOWN,-1,-1
2016-12-20,DOWN,-1,0


#### Add conditions to the strategy

In [52]:
class ClassificationUP(Strategy):
    def init(self):
        self.model = model_dt

    def next(self):
        explanatory_today = df_explanatory.iloc[[-1],:]
        forecast_tomorrow = model_dt.predict(explanatory_today)[0]
        
        # conditions to sell or buy
        ???

### Define initial conditions

In [54]:
bt = Backtest(data = ???, strategy = ???,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

### Interpret backtesting results

In [59]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],99.873096
Equity Final [$],45536.090993
Equity Peak [$],59377.806997
Return [%],355.36091


Classification Strategy | Regression Strategy
-|-
![](src/res_classification.png) | ![](src/res_regression.png)

Classification Model | Regression Model
-|-
![](src/pred_classification.png) | ![](src/pred_regression.png)